In [1]:
from z3 import *
from itertools import combinations
import time
import numpy as np


# Aux functions

def cumulative(start, duration, resources, total):
    decomposition = []
    for r in resources:
        decomposition.append(
            sum([If(And(start[i] <= r, r < start[i] + duration[i]), resources[i], 0) for i in range(len(start))]) <= total
        )
    return decomposition

def maximum(array):
    maximum = array[0]
    for value in array[1:]:
        maximum = If(value > maximum, value, maximum)
    return maximum

# Instance 
class Instance(object):
    width = 0
    n = 0
    dimensions = []
    def __init__(self, width, n, dimensions):
        self.width = width
        self.n = n
        self.dimensions = dimensions

# Read instances: 
def read_file(file_name):
    dimensions = []
    with open(file_name) as f:
        width = int(f.readline())                 # Width of the plate
        n = int(f.readline())                     # Number of blocks
        while True:
            line = f.readline()
            if not line: 
                break
            dimensions.append(line.split(" "))    # Dimensions of each plate
    for dim in dimensions:
        dim[0] = int(dim[0])
        dim[1] = int(dim[1])
    instance = Instance(width, n, dimensions)
    return instance


# Solve instance: 
def solve(instance):
    optimizer = Optimize()

    # Variable Initalization

    x_dims=[]
    y_dims=[]
    x_coords = []
    y_coords = []

    for x_dim,y_dim in instance.dimensions:
        x_dims.append(x_dim)
        y_dims.append(y_dim)
    
    for block in range(instance.n):
        x_coords.append(Int("x%s" % str(block+1)))
        y_coords.append(Int("y%s" % str(block+1)))

    max_height = math.ceil(sum(y_dims)/(instance.width//max(x_dims)))

    print("Width of the plate: ")
    print(instance.width)
    print("Max height of the plate: ")
    print(max_height)

    # Bounding the domain
    
    optimizer.add([x_coords[block] >= 0 for block in range(instance.n)]) # x domain
    optimizer.add([y_coords[block] >= 0 for block in range(instance.n)]) # y domain
        
    optimizer.add([maximum([x_coords[block] + x_dims[block] for block in range(instance.n)]) <= instance.width]) # fits width
    optimizer.add([maximum([y_coords[block] + y_dims[block] for block in range(instance.n)]) <= max_height]) # fits height
    
    # All Different and Cummulative constraints
    
    optimizer.add([Distinct([x_coords[block]+y_coords[block]]) for block in range(instance.n)])
    optimizer.add(cumulative(y_coords,y_dims,x_dims,instance.width))
    optimizer.add(cumulative(x_coords,x_dims,y_dims,max_height))

    # No overlapping
    
    for (block1,block2) in combinations(range(instance.n),2):
        optimizer.add(Or(x_coords[block1]+x_dims[block1] <= x_coords[block2],
                        x_coords[block2]+x_dims[block2] <= x_coords[block1],
                        y_coords[block1]+y_dims[block1] <= y_coords[block2],
                        y_coords[block2]+y_dims[block2] <= y_coords[block1]))
    
    # Symmetry breaking
    
    areas = []
    for block in range(instance.n):
        areas.append(x_dims[block]*y_dims[block])
    
    block_biggest_area = np.argmax(np.asarray(areas))
    optimizer.add([And(x_coords[block_biggest_area] == 0, y_coords[block_biggest_area] == 0)])
    
    # Goal: minimize height
    
    real_height = maximum([y_coords[block] + y_dims[block] for block in range(instance.n)])
    
    optimizer.minimize(real_height)

    timeout = 500 * 1000
    
    if optimizer.check() == sat:
        model = optimizer.model()
        solution_height = model.evaluate(real_height).as_string()
        print("Satisfiable")
        print(solution_height)
    else:
        print("Not satisfiable")
        
    return optimizer

def main():
    instance_file = "..\instances\ins-5.txt"
    instance = read_file(instance_file)

    start = time.time()
    optimizer = solve(instance)
    end = time.time()

    print("{:.2f}".format(end - start) + " seconds")


if __name__ == '__main__':
    main()

Width of the plate: 
12
Max height of the plate: 
23
Satisfiable
12
0.17 seconds


In [12]:
x = Bool("x")
y = Bool("y")
z = Bool("z")
at_most_one([x,y,z])

TypeError: at_most_one() missing 1 required positional argument: 'name'

In [77]:
x = [1,2,3]
x[1:3]

[2, 3]